In [164]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [165]:
import tensorflow as tf
import numpy as np
import json
from time import time

In [166]:
glove_dimension = 50
glove_dimension = 50
def glove_dictionary(file_path, glove_d):
    # select appropriate glove file for glove dimension given
    glove_vectors = open(file_path)    

    length = 0
    vocab = []
    dictionary = {}
    for line in glove_vectors:
        item = line.split(" ")
        vocab.append(item[0])
        word_vector = [float(n) for n in item[1:]]
        dictionary[item[0]] = word_vector
        length = length + 1
        #if length > 10:
         #   break
    
    glove_vectors.close()
    return dictionary

word_dictionary = glove_dictionary("glove.6B/glove.6B.50d.txt", glove_dimension)



def glove(word, dictionary, glove_d):
    if word in dictionary:
        return  dictionary[word]
    else:
        #values taken from paper
        return np.random.uniform(-0.05, 0.05, glove_d).tolist()

print(glove("the", word_dictionary, glove_dimension))



[0.418, 0.24968, -0.41242, 0.1217, 0.34527, -0.044457, -0.49688, -0.17862, -0.00066023, -0.6566, 0.27843, -0.14767, -0.55677, 0.14658, -0.0095095, 0.011658, 0.10204, -0.12792, -0.8443, -0.12181, -0.016801, -0.33279, -0.1552, -0.23131, -0.19181, -1.8823, -0.76746, 0.099051, -0.42125, -0.19526, 4.0071, -0.18594, -0.52287, -0.31681, 0.00059213, 0.0074449, 0.17778, -0.15897, 0.012041, -0.054223, -0.29871, -0.15749, -0.34758, -0.045637, -0.44251, 0.18785, 0.0027849, -0.18411, -0.11514, -0.78581]


In [167]:
def get_data(file, max_num):
    # I reccomend that the max_num is kept very small as my padding function is very slow 
    i = 0
    data = open(file)
    SNLI_sentences = []
    for pair in data:
        pair = json.loads(pair)          # pair will be string so need json.loads to turn it into a dictionary
        if pair["gold_label"] == "-":
            continue                            #get rid of useless results
        SNLI_sentences.append([pair["sentence1"].lower(), pair["sentence2"].lower(), pair["gold_label"]])

        i = i + 1
        if i > max_num:
            break
    # close open file to free up memory
    data.close()

    return SNLI_sentences

# sentence_list is list of lists where the inner lists are [Premise, Hypothesis, label]
sentence_list = get_data("snli_1.0/snli_1.0_train.jsonl", 1000000)
test_list = get_data("snli_1.0/snli_1.0_test.jsonl", 500000)



#print(sentence_list)

def sentence_vectoriser(sentence):
    sentence_array = [glove(item, word_dictionary, glove_dimension) for item in sentence.split(" ")]
    return sentence_array

def OHencoder(tensor):
    # manually encoding labels. if there were more classes then could use SKlearn's one hot encoder but for 3 classes
    # this seems like overkill
    #label = tensor[i][2]
    enc_dict = {"entailment": [1,0,0], "neutral":[0,1,0], "contradiction":[0,0,1]}
    #loop backwards here so that when deleting items it doesn't skip forward
    #or i in range(len(tensor)):
    n = len(tensor)
    i = 0
    while i < n:
        label = tensor[-i][2]
        try:
            tensor[-i][2] = enc_dict[label]
        except KeyError:      
            del tensor[-i]
            n = n - 1
        except TypeError:
            del tensor[-i]
            n = n - 1
        i = i + 1
        """ label in enc_dict:
            tensor[-i][2] = enc_dict[label]
            
        else:
            del tensor[-i]"""
    return tensor
            
        

def pad_sentence_lengths(tensor):
    max_length = 0
    for sen_arr in tensor:
        max_length = max([max_length, len(sen_arr)])
    
    padding = [0 for a in range(glove_dimension)]
    
    for i in range(len(tensor)):
        #tensor[i] = np.lib.pad(tensor[i], (0, max_length - len(tensor[i])), "constant", constant_values = 0)
        for m in range((max_length - len(tensor[i]))):
            tensor[i].append(padding)


    return tensor, max_length


In [ ]:
# This step takes a long time due to the bad padding function

OH_sentence_list = OHencoder(sentence_list)
#train_premises, max_plength = pad_sentence_lengths([sentence_vectoriser(i[0]) for i in OH_sentence_list])
#train_hypothesis, max_hlength = pad_sentence_lengths([sentence_vectoriser(i[1]) for i in OH_sentence_list])
train_labels = [i[2] for i in OH_sentence_list]

tr_prem = [sentence_vectoriser(i[0]) for i in OH_sentence_list]
tr_hyp = [sentence_vectoriser(i[1]) for i in OH_sentence_list]

train_data_length = len(tr_prem)
print(train_data_length)

OH_test_list = OHencoder(test_list)
#test_premises, max_tplength = pad_sentence_lengths([sentence_vectoriser(i[0]) for i in OH_test_list])
#test_hypothesis, max_thlength = pad_sentence_lengths([sentence_vectoriser(i[1]) for i in OH_test_list])
test_labels = [i[2] for i in OH_test_list]

te_prem = [sentence_vectoriser(i[0]) for i in OH_test_list]
te_hyp = [sentence_vectoriser(i[1]) for i in OH_test_list]

test_data_length = len(te_prem)
print(test_data_length)

unpadded_premise_data = tr_prem + te_prem
unpadded_hypothesis_data = tr_hyp + te_hyp
d = train_labels + test_labels
data_labels = np.array(d)
print(len(unpadded_premise_data))
premise_data, max_plength = pad_sentence_lengths(unpadded_premise_data)
hypothesis_data, max_hlength = pad_sentence_lengths(unpadded_hypothesis_data)

print(max_plength)


In [162]:
# I had to define batch size here to be able to run the for loop in the attention function. 
# There is a way that this can be avoided using tf.scan but I ran out of time trying to work out how to do this.
# means when calculating the score of the test set I will have to do this in batches.
batch_size = 50
tf.reset_default_graph()

def length(sequence):
    used = tf.sign(tf.reduce_max(tf.abs(sequence), reduction_indices=2))
    length = tf.reduce_sum(used, reduction_indices=1)
    length = tf.cast(length, tf.int32)
    return length
    


# I havent bothered to implement the linear layer here
LSTM_size = glove_dimension

#input data as 3d tensors
input_premises = tf.placeholder(tf.float32, shape = [batch_size, max_plength, glove_dimension])
input_hypothesis = tf.placeholder(tf.float32, shape = [batch_size, max_hlength, glove_dimension])

#must change scope so that the 2 lstms when run don't have the same variable name
with tf.variable_scope("first_lstm"):
    LSTM_cell = tf.nn.rnn_cell.BasicLSTMCell(LSTM_size, forget_bias = 1.0, state_is_tuple=True)
    p_out, p_state = tf.nn.dynamic_rnn(LSTM_cell, input_premises, dtype=tf.float32, sequence_length=length(input_premises))

#now want p_state tensor to iniatlize the hypothesis lstm
with tf.variable_scope("second_lstm"):
    LSTM_cell = tf.nn.rnn_cell.BasicLSTMCell(LSTM_size, forget_bias = 1.0, state_is_tuple=True)
    h_out, h_state = tf.nn.dynamic_rnn(LSTM_cell, input_hypothesis, initial_state = p_state, dtype=tf.float32, sequence_length=length(input_hypothesis))


# define variables for the attention function
Wy = tf.Variable(tf.truncated_normal([LSTM_size, LSTM_size]))
Wh = tf.Variable(tf.truncated_normal([LSTM_size, LSTM_size]))
Wv = tf.Variable(tf.truncated_normal([LSTM_size, 1]))
Wp = tf.Variable(tf.truncated_normal([LSTM_size, LSTM_size]))
Wx = tf.Variable(tf.truncated_normal([LSTM_size, LSTM_size]))

# h_out will be tensor with shape [batch size, max time, cell.output_size]. make max time first axis using perm
h_out = tf.transpose(h_out, perm = [1, 0 , 2])
last = tf.gather(h_out, int(h_out.get_shape()[0]) - 1)

def attention(H, Y, WY, WH, w, WP, WX):
    b = Y.get_shape()[0]
    seq_l = Y.get_shape()[1]
    #eL = tf.ones([seq_l, 1])
    gather_list = [0 for i in range(seq_l)]
    
    
    for i in range(b):
        Y_i = tf.squeeze(tf.gather(Y, [i]))
        H_i = tf.gather(H, [i])
        #use gather instead of outer product to create the tensor we want
        outer = tf.gather(H_i, gather_list)
        
        M = tf.tanh(tf.matmul(Y_i, WY) + tf.matmul(outer, WH))
        alpha = tf.nn.softmax(tf.matmul(M, w))
        
        r = tf.matmul(alpha, Y_i, transpose_a=True)
        
        h_star = tf.tanh(tf.matmul(r, WP) + tf.matmul(H_i, WX))
        
        if i == 0:
            H_star = h_star
        else:
            H_star = tf.concat(0, [H_star, h_star])
    
    print(H_star.get_shape())
    return H_star
        
    
Att = attention(last, p_out, Wy, Wh, Wv, Wp, Wx)


#final linear layer to get outputs in shape (batch size, 3).
weight = tf.Variable(tf.truncated_normal([LSTM_size, 3]))
bias = tf.Variable(tf.constant(0.1, shape=[3]))
y_ = tf.placeholder(tf.float32, shape = [batch_size, 3])


y = tf.matmul(Att, weight) + bias
#softmax to get predictions
cross_entropy = -tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y, y_))

#minimise using the ADAM optimiser
optimizer = tf.train.AdamOptimizer()
minimize = optimizer.minimize(cross_entropy)

# now work out how many the model gets right
correct = tf.equal(tf.argmax(y_, 1), tf.argmax(y, 1))
num_correct = tf.reduce_sum(tf.cast(correct, tf.float32))
    

(50, 50)


In [163]:
init_op = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init_op)

no_of_batches = int(float(train_data_length)/batch_size)
ptr = 0
for j in range(no_of_batches):
    inp_p, inp_h, out = premise_data[ptr:ptr+batch_size], hypothesis_data[ptr:ptr+batch_size], data_labels[ptr:ptr+batch_size]
    ptr += batch_size
    sess.run(minimize,{input_premises: inp_p, input_hypothesis: inp_h, y_: out})

right = sess.run(num_correct, {input_premises: inp_p, input_hypothesis: inp_h, y_: out})
pt = train_data_length
cumu_right = 0
no_test_batches = int(test_data_length/batch_size)
for l in range(no_test_batches):
    test_p, test_h, test_l = premise_data[pt:pt+batch_size], hypothesis_data[pt:pt+batch_size], data_labels[pt:pt+batch_size]
    right = sess.run(num_correct, {input_premises: test_p, input_hypothesis: test_h, y_: test_l})
    cumu_right = cumu_right + right
accuracy = 100 * cumu_right/test_data_length
print("accuracy is {0} percent".format(accuracy))

# To get the same results as the paper remember to add the linear layer. 
# To increase speed put the sentences into buckets rather than one large data set 
# 
# 
# 
# 


accuracy is 29.940119760479043 percent
